Correct Extra data in course sullabus


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Filter function
def is_valid_line(text):
    text_lower = text.lower()
    forbidden_keywords = [
        '?', 'integrative question', 'certification', 'companies using', 'history of',
        'curriculum', 'career scope', 'steps to learn', 'expertise of', 'placement record',
        'industrial demand', 'overview of', 'what is embedded', 'why choose', 'cetpa',
        'frequently asked', 'faq', 'join best', 'commercial use', '1970', '1960',
        '1940', 'apollo', 'microprocessor', 'transistor', 'miniature circuit',
    ]
    return not any(keyword in text_lower for keyword in forbidden_keywords)

# Load course links from Excel
input_file = "C:\\Users\\taslim.siddiqui\\Downloads\\cepta_course_link.xlsx"
df_links = pd.read_excel(input_file)
course_links = df_links['Course Link'].dropna().tolist()

# Output results
output_data = []

# Setup headless browser
chrome_options = Options()
chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=chrome_options)

# Loop through each course link
for course_link in course_links:
    driver.get(course_link)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Course Name
    course_name_tag = soup.find('h2', class_='titleCertificate')
    if not course_name_tag:
        course_name_tag = soup.find('h1', class_='entry-titles thim-text-title')

    if course_name_tag:
        course_name = course_name_tag.get_text(strip=True)
        course_name = re.sub(r'\bin delhi ncr\b|\bbest\b', '', course_name, flags=re.IGNORECASE).strip()
    else:
        course_name = "Course Name: Not found"

    # About Course
    about_course_paragraphs = soup.find_all('p', class_='pra')
    about_course = '\n'.join(p.get_text(strip=True) for p in about_course_paragraphs if p.get_text(strip=True))

    # Certificate Image (Static link)
    certificate_img_url = "https://www.cetpainfotech.com/wp-content/uploads/2024/07/training-certificate-768x613.png"

    # <td> extraction
    tds = soup.find_all('td')

    # Learning Mode
    learning_modes = []
    for td in tds:
        text = td.get_text(strip=True)
        if any(keyword in text for keyword in ["Instructor-Led", "Virtual", "Classroom"]):
            learning_modes.append(text.replace("*", "").strip())
    learning_mode = ', '.join(sorted(set(learning_modes))) if learning_modes else "Not found"

    # Delivery Mode
    delivery_modes = []
    for td in tds:
        text = td.get_text(strip=True)
        if text in ["*Noida/ *Roorkee/ *Dehradun", "Online"]:
            delivery_modes.append(text)
    delivery_mode = ', '.join(delivery_modes) if delivery_modes else "Not found"

    # Duration Max
    max_durations = []
    for td in tds:
        text = td.get_text(strip=True).lower()
        if re.search(r'\d+(/\d+)*\s*(hours|weeks)', text):
            text = text.replace("*", "").strip()
            if 'weeks' in text:
                week_part = text.split('weeks')[0]
                weeks = re.findall(r'\d+', week_part)
                if weeks:
                    max_weeks = f"{max(map(int, weeks))} Weeks"
                    max_durations.append(max_weeks)
            if 'hours' in text:
                hour_part = text.split('hours')[0]
                hours = re.findall(r'\d+', hour_part)
                if hours:
                    max_hours = f"{max(map(int, hours))} Hours"
                    max_durations.append(max_hours)
    duration_max_str = ", ".join(max_durations) if max_durations else "Not found"

    # Remove unwanted content
    for tag in soup.find_all(['p', 'h2'], class_=['pra', 'keywords']):
        tag.decompose()

    # ✅ Syllabus extraction (correct indentation)
    syllabus_content = ""

    skip_titles = ['overview', 'schedule', 'reviews']
    sections = soup.find_all('span', class_='vc_tta-title-text')
    for section in sections:
        section_title = section.get_text(strip=True)
        if not section_title or not is_valid_line(section_title):
            continue
        syllabus_content += f"\n📘 {section_title}\n"
        panel = section.find_parent('div', class_='vc_tta-panel')
        if not panel:
            continue
        body = panel.find('div', class_='vc_tta-panel-body')
        if not body:
            continue
        for tag in body.find_all(['h3', 'h4', 'ul']):
            if tag.name in ['h3', 'h4'] and tag.get('class') == ['cetpa', 'thim-text-title']:
                module_title = tag.get_text(strip=True)
                if module_title and is_valid_line(module_title):
                    syllabus_content += f"{module_title.upper()}\n"
            elif tag.name == 'ul' and tag.get('class') == ['thim-list-content']:
                for li in tag.find_all('li'):
                    lesson_text = li.get_text(strip=True)
                    if lesson_text and is_valid_line(lesson_text):
                        syllabus_content += f"- {lesson_text}\n"

    # Append result
    output_data.append({
        'Course Name': course_name,
        'Course Link': course_link,
        'About Course': about_course.strip(),
        'Certificate Image': certificate_img_url,
        'Course Learning Mode': learning_mode,
        'Course Delivery Mode': delivery_mode,
        'Duration (Max)': duration_max_str,
        'Syllabus': syllabus_content.strip()
    })

# Close browser
driver.quit()

# Save to Excel
output_path = "C:\\Users\\taslim.siddiqui\\Downloads\\cetpa_final_syllabus1.xlsx"
df_output = pd.DataFrame(output_data)
df_output.to_excel(output_path, index=False)

print("✅ All course data saved to Excel:", output_path)





# -------------------- MAIN --------------------
if __name__ == "__main__":
    course_urls = [
        "https://www.cetpainfotech.com/technology/software-testing#courseContent"
    ]

    print("🚀 Starting scraping process...")
    file_path = r"C:\Users\taslim.siddiqui\Downloads\course00.xlsx"

    for course_url in course_urls:
        print(f"\n🔍 Processing: {course_url}")
        course_data = scrape_course(course_url)
        if all(item != "Error" for item in course_data):
            print(f"\n📖 Course Name: {course_data[0]}")
            print(f"📘 Syllabus:\n{course_data[1]}\n")
            save_to_excel(course_data, file_path, course_url)
        else:
            print(f"❌ Failed to scrape complete data for {course_url}")

    print("\n✅ Process completed")



✅ All course data saved to Excel: C:\Users\taslim.siddiqui\Downloads\cetpa_final_syllabus1.xlsx


Input file correct code

In [33]:
import os
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# -------------------- DRIVER SETUP --------------------
def get_driver(headless=False):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# -------------------- CLEANING --------------------
def clean_text(text):
    if not text:
        return ""
    text = re.sub(r"<[^>]+>", " ", str(text))
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# -------------------- COURSE NAME --------------------
def extract_course_name(soup):
    h2 = soup.find("h2", class_="titleCertificate")
    if h2:
        span = h2.find("span")
        if span and span.get_text(strip=True):
            return clean_text(span.get_text())
        return clean_text(h2.get_text())
    h1 = soup.find("h1")
    if h1:
        return clean_text(h1.get_text())
    return "N/A - Course name not found"

# -------------------- FIND COURSE CONTENT CONTAINER --------------------
def find_course_content_container(soup):
    header = soup.find(lambda tag: tag.name in ["h2", "h3", "h4"]
                       and "cour" in (tag.get("class") or [])
                       and re.search(r"Course\s*Content", tag.get_text(strip=True), re.I))
    if header:
        nxt = header.find_next("div", class_=re.compile(r"\bvc_tta-container\b"))
        if nxt:
            return nxt
    return soup.find("div", class_=re.compile(r"\bvc_tta-container\b"))

# -------------------- SYLLABUS EXTRACTION --------------------
def extract_course_syllabus(soup):
    container = find_course_content_container(soup)
    if not container:
        return "N/A - Syllabus not found"

    lines = []
    seen = set()
    panels = container.find_all("div", class_=re.compile(r"\bvc_tta-panel\b"))

    # exact terms to skip
    skip_exact = {"overview", "certification", "faq"}

    for panel in panels:
        # Panel title
        title_span = panel.find("span", class_="vc_tta-title-text")
        panel_title = clean_text(title_span.get_text(strip=True)) if title_span else ""

        # 🔴 Skip panel if its title is exactly Overview, Certification, or FAQ
        if panel_title.lower() in skip_exact:
            continue

        if panel_title and panel_title not in seen:
            lines.append(panel_title)
            seen.add(panel_title)

        # Panel body content
        body = panel.find("div", class_=re.compile(r"\bvc_tta-panel-body\b"))
        if not body:
            continue

        # Collect h3/h4 headings and nested ULs
        for elem in body.find_all(["h3", "h4", "ul"], recursive=True):
            if elem.name in ["h3", "h4"]:
                heading_text = clean_text(elem.get_text(strip=True))
                # 🔴 Skip heading only if exactly Overview/Certification/FAQ
                if heading_text and heading_text.lower() not in skip_exact:
                    if heading_text not in seen:
                        lines.append(heading_text)
                        seen.add(heading_text)
            elif elem.name == "ul":
                for li in elem.find_all("li"):
                    item_text = clean_text(li.get_text())
                    if item_text and item_text.lower() not in skip_exact:
                        if item_text not in seen:
                            lines.append(f"- {item_text}")
                            seen.add(item_text)

        # MODULE HEADERS (<h3>, <h4>) AND their immediate <ul>/<ol> bullets
        headers = body.find_all(["h3", "h4"])
        for header_tag in headers:
            module_title = clean_text(header_tag.get_text(strip=True))
            if not module_title:
                continue
            if module_title.lower() in skip_exact:
                continue  # skip only if exactly "Overview" etc.

            if module_title not in seen:
                lines.append(module_title)
                seen.add(module_title)

            # Add bullets immediately after this header
            next_sibling = header_tag.find_next_sibling()
            while next_sibling:
                if next_sibling.name in ["h3", "h4"]:
                    break
                if next_sibling.name in ["ul", "ol"]:
                    for li in next_sibling.find_all("li"):
                        t = clean_text(li.get_text())
                        if t and t.lower() not in skip_exact:
                            if t not in seen:
                                lines.append(f"- {t}")
                                seen.add(t)
                    break
                next_sibling = next_sibling.find_next_sibling()

        # Direct thim-list-content bullets
        for ul in body.find_all(["ul", "ol"], class_="thim-list-content"):
            for li in ul.find_all("li"):
                t = clean_text(li.get_text())
                if t and t.lower() not in skip_exact:
                    if t not in seen:
                        lines.append(f"- {t}")
                        seen.add(t)

    return "\n".join(lines) if lines else "N/A - Syllabus not found"

# -------------------- SCRAPER --------------------
def scrape_course(course_url, driver=None, headless=False):
    """
    Returns a dict:
      {
        "Course Name": ...,
        "Course Link": course_url,
        "Course Syllabus": ...
      }
    Reuses provided `driver` if not None, otherwise creates and disposes its own.
    """
    internal_driver = False
    if driver is None:
        driver = get_driver(headless=headless)
        internal_driver = True

    try:
        driver.get(course_url)
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".vc_tta-container, body"))
            )
        except Exception:
            pass
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight/3);")
        time.sleep(1.2)
        html = driver.page_source
    finally:
        if internal_driver:
            driver.quit()

    soup = BeautifulSoup(html, "html.parser")
    course_name = extract_course_name(soup)
    course_syllabus = extract_course_syllabus(soup)
    return {
        "Course Name": course_name,
        "Course Link": course_url,
        "Course Syllabus": course_syllabus
    }

# -------------------- SAVE TO EXCEL --------------------
def save_to_excel(data, file_path, url):
    columns = ["Course Name", "Syllabus", "Course URL"]

    try:
        if os.path.exists(file_path):
            df = pd.read_excel(file_path)
            for col in columns:
                if col not in df.columns:
                    df[col] = None
        else:
            df = pd.DataFrame(columns=columns)

        existing_urls = df["Course URL"].tolist() if "Course URL" in df.columns else []
        if url in existing_urls:
            print(f"🔄 Course already exists: {data[0]}")
            return

        new_row = pd.DataFrame([dict(zip(columns, [*data, url]))])
        df = pd.concat([df, new_row], ignore_index=True)

        df.to_excel(file_path, index=False)
        print(f"💾 Saved data for: {data[0]}")

    except Exception as e:
        print(f"❌ Excel save error: {e}")

# -------------------- MAIN --------------------
def main():
    input_file = r"C:\Users\taslim.siddiqui\Downloads\new code for course syllabus cepta.xlsx"
    output_file = r"C:\Users\taslim.siddiqui\Downloads\CEPTA_course syallbus.xlsx"

    df = pd.read_excel(input_file)
    if "Course Link" not in df.columns:
        print("❌ Excel must have a column named 'Course Link'")
        return

    driver = get_driver(headless=False)
    results = []

    print(f"📖 Found {len(df)} URLs in the Excel file\n")
    for i, url in enumerate(df["Course Link"], 1):
        if pd.isna(url):
            continue
        url = str(url).strip()
        print(f"🔹 Scraping ({i}/{len(df)}): {url}")

        # Now scrape_course returns a dict and reuses the shared driver
        course_data = scrape_course(url, driver=driver)
        results.append(course_data)

        # 🔹 Print Course details (dict-safe)
        if course_data.get("Course Name"):
            print(f"\n📘 Course Name: {course_data['Course Name']}")
            print(f"🔗 Course Link: {course_data['Course Link']}")
        if course_data.get("Course Syllabus"):
            print("\n📚 Course Syllabus:\n")
            print(course_data["Course Syllabus"])
        print("-" * 80)

    driver.quit()

    # Save results to Excel
    out_df = pd.DataFrame(results)
    out_df.to_excel(output_file, index=False)
    print(f"\n💾 All results saved to {output_file}")

if __name__ == "__main__":
    main()


📖 Found 178 URLs in the Excel file

🔹 Scraping (1/178): https://www.cetpainfotech.com/technology/my-sql-training

📘 Course Name: MySQL Certification Training
🔗 Course Link: https://www.cetpainfotech.com/technology/my-sql-training

📚 Course Syllabus:

Introduction To MYSQL
Installing MYSQL On Windows And Linux
MYSQL configuration
CREATING ACCOUNT, CHANGING ROOT PASSWORD AND DELETING ANONYMOUS ACCOUNTS
UNDERSTANDING VARIOUS MYSQL EXECUTABLE FILES
MYSQL User Interfaces
MYSQL MONITORING
CREATING/DELETING DATABASES
MYSQL Column Type Databasem
IDENTIFIERS & QUALIFIERS
OPERATORS & FUNCTIONS
MYSQL String Function
SELECT STATEMENT
SELECTING ALL COLUMNS OR SPECIFIC COLUMNS
Data Definition Statements
DATA MANIPULATION STATEMENTS
JOINS
Views
WRITING SUBQUERIES
LOCKING ISSUES
MYSQL Transactional And Locking Statements
ACCOUNT MANAGEMENT STATEMENTS
TABLE MAINTENANCE STATEMENTS
SET Syntax
MYSQL Backup
MYSQL Replication
--------------------------------------------------------------------------------
🔹

Do for one course link

In [ ]:
import os
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# -------------------- DRIVER SETUP --------------------
def get_driver(headless=False):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# -------------------- CLEANING --------------------
def clean_text(text):
    if not text:
        return ""
    text = re.sub(r"<[^>]+>", " ", str(text))
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# -------------------- COURSE NAME --------------------
def extract_course_name(soup):
    h2 = soup.find("h2", class_="titleCertificate")
    if h2:
        span = h2.find("span")
        if span and span.get_text(strip=True):
            return clean_text(span.get_text())
        return clean_text(h2.get_text())
    h1 = soup.find("h1")
    if h1:
        return clean_text(h1.get_text())
    return "N/A - Course name not found"

# -------------------- FIND COURSE CONTENT CONTAINER --------------------
def find_course_content_container(soup):
    header = soup.find(lambda tag: tag.name in ["h2", "h3", "h4"]
                       and "cour" in (tag.get("class") or [])
                       and re.search(r"Course\s*Content", tag.get_text(strip=True), re.I))
    if header:
        nxt = header.find_next("div", class_=re.compile(r"\bvc_tta-container\b"))
        if nxt:
            return nxt
    return soup.find("div", class_=re.compile(r"\bvc_tta-container\b"))

# -------------------- SYLLABUS EXTRACTION --------------------
def extract_course_syllabus(soup):
    container = find_course_content_container(soup)
    if not container:
        return "N/A - Syllabus not found"

    lines = []
    seen = set()
    panels = container.find_all("div", class_=re.compile(r"\bvc_tta-panel\b"))

    # exact terms to skip
    skip_exact = {"overview", "certification", "faq"}

    for panel in panels:
        # Panel title
        title_span = panel.find("span", class_="vc_tta-title-text")
        panel_title = clean_text(title_span.get_text(strip=True)) if title_span else ""

        # 🔴 Skip panel if its title is exactly Overview, Certification, or FAQ
        if panel_title.lower() in skip_exact:
            continue

        if panel_title and panel_title not in seen:
            lines.append(panel_title)
            seen.add(panel_title)

        # Panel body content
        body = panel.find("div", class_=re.compile(r"\bvc_tta-panel-body\b"))
        if not body:
            continue

        # Collect h3/h4 headings and nested ULs
        for elem in body.find_all(["h3", "h4", "ul"], recursive=True):
            if elem.name in ["h3", "h4"]:
                heading_text = clean_text(elem.get_text(strip=True))
                # 🔴 Skip heading only if exactly Overview/Certification/FAQ
                if heading_text and heading_text.lower() not in skip_exact:
                    if heading_text not in seen:
                        lines.append(heading_text)
                        seen.add(heading_text)
            elif elem.name == "ul":
                for li in elem.find_all("li"):
                    item_text = clean_text(li.get_text())
                    if item_text and item_text.lower() not in skip_exact:
                        if item_text not in seen:
                            lines.append(f"- {item_text}")
                            seen.add(item_text)

        # MODULE HEADERS (<h3>, <h4>) AND their immediate <ul>/<ol> bullets
        headers = body.find_all(["h3", "h4"])
        for header_tag in headers:
            module_title = clean_text(header_tag.get_text(strip=True))
            if not module_title:
                continue
            if module_title.lower() in skip_exact:
                continue  # skip only if exactly "Overview" etc.

            if module_title not in seen:
                lines.append(module_title)
                seen.add(module_title)

            # Add bullets immediately after this header
            next_sibling = header_tag.find_next_sibling()
            while next_sibling:
                if next_sibling.name in ["h3", "h4"]:
                    break
                if next_sibling.name in ["ul", "ol"]:
                    for li in next_sibling.find_all("li"):
                        t = clean_text(li.get_text())
                        if t and t.lower() not in skip_exact:
                            if t not in seen:
                                lines.append(f"- {t}")
                                seen.add(t)
                    break
                next_sibling = next_sibling.find_next_sibling()

        # Direct thim-list-content bullets
        for ul in body.find_all(["ul", "ol"], class_="thim-list-content"):
            for li in ul.find_all("li"):
                t = clean_text(li.get_text())
                if t and t.lower() not in skip_exact:
                    if t not in seen:
                        lines.append(f"- {t}")
                        seen.add(t)

    return "\n".join(lines) if lines else "N/A - Syllabus not found"

# -------------------- SCRAPER --------------------
def scrape_course(course_url, headless=False):
    driver = get_driver(headless=headless)
    try:
        driver.get(course_url)
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".vc_tta-container, body"))
            )
        except Exception:
            pass
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight/3);")
        time.sleep(1.2)
        html = driver.page_source
    finally:
        driver.quit()

    soup = BeautifulSoup(html, "html.parser")
    course_name = extract_course_name(soup)
    course_syllabus = extract_course_syllabus(soup)
    return [course_name, course_syllabus]

# -------------------- SAVE TO EXCEL --------------------
def save_to_excel(data, file_path, url):
    columns = ["Course Name", "Syllabus", "Course URL"]

    try:
        if os.path.exists(file_path):
            df = pd.read_excel(file_path)
            for col in columns:
                if col not in df.columns:
                    df[col] = None
        else:
            df = pd.DataFrame(columns=columns)

        existing_urls = df["Course URL"].tolist() if "Course URL" in df.columns else []
        if url in existing_urls:
            print(f"🔄 Course already exists: {data[0]}")
            return

        new_row = pd.DataFrame([dict(zip(columns, [*data, url]))])
        df = pd.concat([df, new_row], ignore_index=True)

        df.to_excel(file_path, index=False)
        print(f"💾 Saved data for: {data[0]}")

    except Exception as e:
        print(f"❌ Excel save error: {e}")

# -------------------- MAIN --------------------
if __name__ == "__main__":
    course_urls = [
        "https://www.cetpainfotech.com/technology/vhdl#courseContent"
    ]

    print("🚀 Starting scraping process...")
    file_path = r"C:\Users\taslim.siddiqui\Downloads\course00.xlsx"

    for course_url in course_urls:
        print(f"\n🔍 Processing: {course_url}")
        course_data = scrape_course(course_url)
        if all(item != "Error" for item in course_data):
            print(f"\n📖 Course Name: {course_data[0]}")
            print(f"📘 Syllabus:\n{course_data[1]}\n")
            save_to_excel(course_data, file_path, course_url)
        else:
            print(f"❌ Failed to scrape complete data for {course_url}")

    print("\n✅ Process completed")


🚀 Starting scraping process...

🔍 Processing: https://www.cetpainfotech.com/technolgy/data-science-using-python#courseContent

📖 Course Name: Data Science Using Python Certification Training
📘 Syllabus:
Curriculum Of Core & Advanced Python
GETTING STARTED
- History & need of Python
- Application of Python
- Advantages of Python
- Disadvantages of Python
- Installing Python
- Program structure
- Interactive Shell
- Executable or script files.
- User Interface or IDE
Python Fundamentals
DATA HANDLING
- Data Types
- Numbers
- Strings
- Lists
- Tuples
- Dictionary
- Set
- Frozenset
- Bool
- Mutable and Immutable
STRING MANIPULATION
- Introduction to Python String
- Accessing Individual Elements
- String Operators
- String Slices
- String Functions and Methods
List Manipulation
TUPLES
- Introduction to Tuple
- Creating Tuples
- Accessing Tuples
- Joining Tuples
- Replicating Tuples
- Tuple Slicing
DICTIONARIES
- Introduction to Dictionary
- Accessing values in dictionaries
- Working with di